In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torch.utils import data
import torch
import os
from from_scratch.dataset import Dataset
from from_scratch.losses import SimpleLoss
from from_scratch.model import Yolo
from from_scratch.inference import predict

In [7]:
# saving checkpoint
torch.save({
    'epoch': 70, # 61 1e-3, 9 1e-4
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': epoch_loss / len(train_loader)
}, 'yolo_n_checkpoint.pth')

# Load the model
#model = Yolo(version='n')
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
#checkpoint = torch.load('yolo_n_checkpoint.pth')
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#print(f"Loaded model from checkpoint at epoch {checkpoint['epoch']} with loss {checkpoint['loss']:.4f}")

In [3]:
dataset = Dataset('/home/davide/Desktop/change_detection/dataset', input_size=640)
train_loader = data.DataLoader(dataset, batch_size=32, num_workers=0, pin_memory=True, shuffle=True)
print(f"Train_loader : {len(train_loader)} batches")

Train_loader : 4 batches


In [5]:
torch.manual_seed(42)

model = Yolo(version='n')
print(f"{sum(p.numel() for p in model.parameters())/1e6} million parameters")

criterion = SimpleLoss(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

num_epochs = 30

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for imgs, targets in train_loader:
        imgs = imgs.float()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch: {epoch + 1} | avg loss: {epoch_loss / len(train_loader)}")

2.629046 million parameters
Epoch: 1 | avg loss: 2.013804181274094
Epoch: 2 | avg loss: 1.7436317665845649
Epoch: 3 | avg loss: 1.6028307106015538
Epoch: 4 | avg loss: 1.5368230259860436
Epoch: 5 | avg loss: 1.4677248973116397
Epoch: 6 | avg loss: 1.4044563858608656
Epoch: 7 | avg loss: 1.3461342623187806
Epoch: 8 | avg loss: 1.290515179876036
Epoch: 9 | avg loss: 1.2412014812806489
Epoch: 10 | avg loss: 1.184950146466404
Epoch: 11 | avg loss: 1.1315944795396284
Epoch: 12 | avg loss: 1.0768450068210467
Epoch: 13 | avg loss: 1.0213204888243799
Epoch: 14 | avg loss: 0.9839255352815224
Epoch: 15 | avg loss: 0.9376639131405664
Epoch: 16 | avg loss: 0.8914764861828213
Epoch: 17 | avg loss: 0.8448672136776252
Epoch: 18 | avg loss: 0.7973462979327675
Epoch: 19 | avg loss: 0.7525358553242576
Epoch: 20 | avg loss: 0.7140711768550606
Epoch: 21 | avg loss: 0.6722284644479877
Epoch: 22 | avg loss: 0.6373808829789234
Epoch: 23 | avg loss: 0.6014856045099767
Epoch: 24 | avg loss: 0.5654010201866859


In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
#criterion = SimpleLoss(model)
num_epochs = 10

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for imgs, targets in train_loader:
        imgs = imgs.float()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch : {epoch + 1 + 70} | avg loss : {epoch_loss / len(train_loader)}")

Epoch : 51 | avg loss : 0.060591915971553556
Epoch : 52 | avg loss : 0.058168523032657765
Epoch : 53 | avg loss : 0.05648197227675843
Epoch : 54 | avg loss : 0.05476493010019803
Epoch : 55 | avg loss : 0.055671591371285425
Epoch : 56 | avg loss : 0.055174788261549895
Epoch : 57 | avg loss : 0.05427632030132176
Epoch : 58 | avg loss : 0.0533780261644389
Epoch : 59 | avg loss : 0.05289243286676379
Epoch : 60 | avg loss : 0.05328700516006177


In [5]:
test_images = os.listdir('/home/davide/Desktop/change_detection/dataset/images/val')

for image_name in test_images:
    image_path = os.path.join('/home/davide/Desktop/change_detection/dataset/images/val', image_name)
    out = predict(model, image_path, scale=20, threshold=0.05)
    out += predict(model, image_path, scale=40, threshold=0.05)
    out += predict(model, image_path, scale=80, threshold=0.05)
    out[out > 1] = 1
    plt.imsave(f'/home/davide/Desktop/change_detection/from_scratch/predictions/{image_name}', out, cmap='gray')